<a href="https://colab.research.google.com/github/cbaldassari/manifold/blob/main/manifold_step1_wass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install --upgrade xlrd
#!pip install missingpy==0.2.0
#!pip uninstall scikit-learn
#pip install scikit-learn==0.22.2.post1

In [ ]:
%%capture
!pip install scikit-learn==1.1.2
!pip install scipy==1.9.1
!pip install missingpy==0.2.0
!pip install yfinance

In [ ]:
import os
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
from sklearn import mixture
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import kurtosis
from tqdm import tqdm
from missingpy import MissForest
import yfinance as yf

In [ ]:
yahoo = yf.download('SPY', start='2018-01-01', end='2022-12-31')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
df1=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/ice_electric-2018final.xlsx")
df2=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/ice_electric-2019final.xlsx")
df3=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/ice_electric-2020final.xlsx")
df4=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/ice_electric-2021final.xlsx")
df5=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/ice_electric-2022final.xlsx")

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5], ignore_index=True, sort=False)
elec=df[["Price hub","Trade date","Wtd avg price $/MWh"]]

In [ ]:
dfe1=elec[elec['Price hub']=="PJM WH Real Time Peak"]
dfe1["hub"]="pjm"

dfe2=elec[elec['Price hub']=="Palo Verde Peak"]
dfe2["hub"]="paloverde"

dfe3=elec[elec['Price hub']=="SP15 EZ Gen DA LMP Peak"]
dfe3["hub"]="sp15"

dfe4=elec[elec['Price hub']=="Nepool MH DA LMP Peak"]
dfe4["hub"]="nepool"

dfe5=elec[elec['Price hub']=="Mid C Peak"]
dfe5["hub"]="midcolumbia"

#dfe6=elec[elec['Price hub']=="PJM WH Real Time Peak"]
#dfe6["hub"]="pjm"

dfe = pd.concat([dfe1,dfe2,dfe3,dfe4,dfe5], ignore_index=True, sort=False)

<ipython-input-7-6773057c8296>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe1["hub"]="pjm"
<ipython-input-7-6773057c8296>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe2["hub"]="paloverde"
<ipython-input-7-6773057c8296>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

##missForest

##Adapt different series columns' name

In [ ]:
nepool=dfe[dfe["hub"]=="nepool"]
nepool.reset_index(inplace=True)
nepool.drop(columns=["Price hub","hub","index"],inplace=True)
nepool.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
nepool

<ipython-input-8-d2443199ca92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nepool.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-8-d2443199ca92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nepool.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,199.71
1,2018-01-04 00:00:00,311.14
2,2018-01-05 00:00:00,149.08
3,2018-01-09 00:00:00,89.25
4,2018-01-10 00:00:00,53.50
...,...,...
1106,2022-12-19 00:00:00,135.83
1107,2022-12-20 00:00:00,110.67
1108,2022-12-21 00:00:00,81.68
1109,2022-12-22 00:00:00,209.63


In [ ]:
pjm=dfe[dfe["hub"]=="pjm"]
pjm.reset_index(inplace=True)
pjm.drop(columns=["Price hub","hub","index"],inplace=True)
pjm.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
pjm

<ipython-input-9-83127bb27ac2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pjm.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-9-83127bb27ac2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pjm.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,172.81
1,2018-01-04 00:00:00,366.91
2,2018-01-05 00:00:00,79.43
3,2018-01-09 00:00:00,36.31
4,2018-01-10 00:00:00,29.31
...,...,...
1221,2022-12-19 00:00:00,85.45
1222,2022-12-20 00:00:00,79.86
1223,2022-12-21 00:00:00,69.11
1224,2022-12-22 00:00:00,181.75


In [ ]:
sp15=dfe[dfe["hub"]=="sp15"]
sp15.reset_index(inplace=True)
sp15.drop(columns=["Price hub","hub","index"],inplace=True)
sp15.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
sp15

<ipython-input-13-a4e1f6993e0b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp15.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-13-a4e1f6993e0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp15.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,50.96
1,2018-01-04 00:00:00,36.42
2,2018-01-05 00:00:00,37.26
3,2018-01-09 00:00:00,33.00
4,2018-01-10 00:00:00,36.70
...,...,...
1172,2022-12-19 00:00:00,320.10
1173,2022-12-20 00:00:00,277.38
1174,2022-12-21 00:00:00,365.77
1175,2022-12-22 00:00:00,190.00


In [ ]:
paloverde=dfe[dfe["hub"]=="paloverde"]
paloverde.reset_index(inplace=True)
paloverde.drop(columns=["Price hub","hub","index"],inplace=True)
paloverde.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
paloverde

<ipython-input-10-eaec451fa2be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paloverde.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-10-eaec451fa2be>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paloverde.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,42.78
1,2018-01-04 00:00:00,29.80
2,2018-01-05 00:00:00,26.28
3,2018-01-09 00:00:00,22.01
4,2018-01-10 00:00:00,23.69
...,...,...
1225,2022-12-19 00:00:00,323.14
1226,2022-12-20 00:00:00,275.27
1227,2022-12-21 00:00:00,320.22
1228,2022-12-22 00:00:00,181.83


In [ ]:
spx=pd.DataFrame(yahoo["Close"])
spx.reset_index(inplace=True)
spx.rename(columns={"Close":'Price'}, inplace=True)
spx

,Date,Price
0,2018-01-02,268.769989
1,2018-01-03,270.470001
2,2018-01-04,271.609985
3,2018-01-05,273.420013
4,2018-01-08,273.920013
...,...,...
1254,2022-12-23,382.910004
1255,2022-12-27,381.399994
1256,2022-12-28,376.660004
1257,2022-12-29,383.440002


In [ ]:
oil=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/oil.xls",sheet_name="Data 1", parse_dates=True)

In [ ]:
#Timeseriesdatacanbedownloadedfromwww.eia.gov/electricity/wholesale(PJM),www.eia.gov/dnav/ng/hist/rngwhhdm.htm(GAS),www.eia.gov/dnav/ pet/hist/rwtcD.htm(OIL)andwww.macrotrends.net(SPX),thelatteroneisfee-based.

In [ ]:
gas=pd.read_excel("/content/drive/MyDrive/Mari/Paper6/data/gas.xls",sheet_name="Data 1", parse_dates=True)

In [ ]:
terms=pd.read_excel("/content/drive/MyDrive/Mari/Paper5/data/LW_daily.xlsx", parse_dates=["Date"])

In [ ]:
terms.set_index("Date")

,1 m,2 m,3 m,4 m,5 m,6 m,7 m,8 m,9 m,10 m,...,351 m,352 m,353 m,354 m,355 m,356 m,357 m,358 m,359 m,360 m
Date,,,,,,,,,,,,,,,,,,,,,
1961-06-14,2.245463,2.312255,2.381113,2.451428,2.522306,2.592520,2.660420,2.723988,2.781463,2.832570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-06-15,2.238297,2.305464,2.373091,2.441878,2.511999,2.582966,2.653560,2.721918,2.785903,2.843929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-06-16,2.235452,2.303258,2.371947,2.442190,2.513979,2.586526,2.658266,2.727051,2.790648,2.847660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-06-19,2.217626,2.295898,2.371827,2.446512,2.520499,2.593621,2.664923,2.732801,2.795486,2.851950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-06-20,2.182799,2.270473,2.353695,2.433896,2.511963,2.588052,2.661447,2.730642,2.793852,2.850058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,3.867337,4.023574,4.154775,4.259353,4.338186,4.394684,4.433878,4.460945,4.480016,4.493783,...,3.816482,3.813802,3.811096,3.808366,3.805612,3.802834,3.800036,3.797223,3.794403,3.791584
2022-12-27,3.903516,4.073273,4.213900,4.324662,4.407392,4.466296,4.506835,4.534270,4.552698,4.564882,...,3.919447,3.916438,3.913439,3.910452,3.907474,3.904505,3.901540,3.898580,3.895626,3.892682
2022-12-28,3.995884,4.136985,4.255885,4.351028,4.423119,4.474977,4.510701,4.534563,4.550189,4.560310,...,3.953790,3.950572,3.947351,3.944128,3.940907,3.937687,3.934469,3.931256,3.928051,3.924859


In [ ]:
term1m=terms[["Date","  1 m"]].copy(deep=True)
term1m.rename(columns={"  1 m":"Price"},inplace=True)

In [ ]:
term6m=terms[["Date","  6 m"]].copy(deep=True)
term6m.rename(columns={"  6 m":"Price"},inplace=True)

In [ ]:
term1Yr=terms[["Date"," 12 m"]].copy(deep=True)
term1Yr.rename(columns={" 12 m":"Price"},inplace=True)

In [ ]:
term5Yr=terms[["Date"," 60 m"]].copy(deep=True)
term5Yr.rename(columns={" 60 m":"Price"},inplace=True)

In [ ]:
term7Yr=terms[["Date"," 84 m"]].copy(deep=True)
term7Yr.rename(columns={" 84 m":"Price"},inplace=True)

In [ ]:
term10Yr=terms[["Date","120 m"]].copy(deep=True)
term10Yr.rename(columns={"120 m":"Price"},inplace=True)

In [ ]:
term20Yr=terms[["Date","240 m"]].copy(deep=True)
term20Yr.rename(columns={"240 m":"Price"},inplace=True)

In [ ]:
multi_ts_dic={
    "sp15":sp15,
    "nepool":nepool,
    "paloverde":paloverde,
    "pjm":pjm,
    "spx":spx,
    "gas": gas,
    "oil": oil,
        "term1m":term1m,
        "term6m":term6m,
    "term1Yr":term1Yr,
    "term5Yr":term5Yr,
    "term7Yr":term7Yr,
    "term10Yr":term10Yr,
    "term20Yr":term20Yr}

In [ ]:
stocha=[]
hubfilled=pd.DataFrame()
stochastic2=pd.DataFrame()
stochastic3=pd.DataFrame()
stochastic4=pd.DataFrame()
logprezzi=pd.DataFrame()
trend=pd.DataFrame()
prezzi=pd.DataFrame()
flagin=True
chart=pd.DataFrame()
logprezzidepurati=pd.DataFrame()
logprezzi=pd.DataFrame()
filLogPrices=pd.DataFrame()
stochastic=pd.DataFrame()
x=pd.DataFrame()
r=pd.DataFrame()
linechart=pd.DataFrame()
prezzifillati=pd.DataFrame()

In [ ]:
%%capture
for hub in multi_ts_dic:
  print(hub)
  huball=multi_ts_dic[hub]
  #huball.drop(columns=['Price hub'], inplace=True)
  huball.rename(columns={'Wtd avg price $/MWh':'Price',  'Trade date':'Date'}, inplace=True)
  print(huball.Price[huball.Price<=0])
  huball.Price[huball.Price<=0]=np.nan
  print(huball.Price[huball.Price<=0])
  lowess=pd.DataFrame()

  #huballhub=huball[huball['hub_transcode'] == hub]      #ritaglio il singolo hub -> missForest cambierebbe
  huball=huball.groupby('Date').first() #group by per record doppi -> prendo il primo

  #Creo la griglia e sovrappongo la serie dei prezzi
  start = '01-01-2018'
  end = '31-12-2022'
  grid=pd.Series(pd.date_range(start = start, end = end, freq='D'))
  grid.sort_index(ascending=True)
  huball= huball.reindex(grid, fill_value=np.nan)
  huball.index.name = "Date"

  linechart[hub]=huball["Price"]

  toimpute=pd.DataFrame()
  toimpute["Date"]=np.arange(len(huball))
  toimpute["Price"]=huball["Price"].values

  imputer = MissForest(random_state=1234)
  X_imputed = imputer.fit_transform(toimpute.values)

  filLogPrices[hub]=np.log(X_imputed[:,1])
  prezzifillati[hub]=X_imputed[:,1]
  trend[hub] = sm.nonparametric.lowess(filLogPrices[hub], filLogPrices.index, frac=0.1, it=10, is_sorted=True, return_sorted=False, missing="raise")
  stochastic[hub] = filLogPrices[hub]-trend[hub].values

  logprezzidepurati[hub]=stochastic[hub]
  logprezzi[hub]=filLogPrices[hub]

  x[hub]=stochastic[hub]
  r[hub]=stochastic[hub]-stochastic[hub].shift(1)
  print("xxxxxx",x.shape)
  print("rrrrrr",r.shape)

filLogPrices["Date"]=huball.index
filLogPrices.set_index("Date", inplace=True)

logprezzidepurati["Date"]=huball.index
logprezzidepurati.set_index("Date", inplace=True)

logprezzi["Date"]=huball.index
logprezzi.set_index("Date", inplace=True)

x["Date"]=huball.index
x.set_index("Date", inplace=True)

r["Date"]=huball.index
r.set_index("Date", inplace=True)
r.dropna(inplace=True)


In [ ]:
x.to_excel("/content/drive/MyDrive/Mari/Paper6/data/chartData/x.xlsx")
r.to_excel("/content/drive/MyDrive/Mari/Paper6/data/chartData/r.xlsx")
filLogPrices.to_excel("/content/drive/MyDrive/Mari/Paper6/data/chartData/filLogPrices.xlsx")
logprezzidepurati.to_excel("/content/drive/MyDrive/Mari/Paper6/data/chartData/logprezzidepurati.xlsx")
trend.to_excel("/content/drive/MyDrive/Mari/Paper6/data/chartData/trend.xlsx")

In [ ]:
x

,sp15,nepool,paloverde,pjm,spx,gas,oil,term1m,term6m,term1Yr,term5Yr,term7Yr,term10Yr,term20Yr
Date,,,,,,,,,,,,,,
2018-01-01,0.178903,0.863926,0.327492,1.887689,-0.026625,0.734302,-0.013875,-0.006274,-0.004077,-0.005590,-0.053734,-0.058816,-0.057051,-0.047833
2018-01-02,0.181305,0.873947,0.328729,1.885954,-0.029136,0.777495,-0.021651,-0.008265,-0.006574,-0.007886,-0.056294,-0.059379,-0.056180,-0.045450
2018-01-03,0.282380,0.810724,0.453781,1.737747,-0.022487,0.778270,-0.001814,-0.017746,-0.009748,-0.009863,-0.058952,-0.066109,-0.065008,-0.056203
2018-01-04,-0.051138,1.264082,0.093452,2.488923,-0.017938,0.484925,0.003675,-0.010586,-0.009319,-0.008654,-0.050116,-0.060850,-0.065002,-0.058535
2018-01-05,-0.025928,0.538284,-0.031018,0.956927,-0.010953,0.603917,-0.004764,-0.017881,-0.010688,-0.009998,-0.042716,-0.052507,-0.056157,-0.050316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.686069,0.950367,0.639204,0.318043,-0.017073,-0.026481,0.031532,-0.103658,-0.072438,-0.072201,-0.016692,-0.006307,-0.002817,0.003048
2022-12-28,0.722203,0.956225,0.676011,0.547667,-0.029541,-0.206865,0.025936,-0.084963,-0.073428,-0.075608,-0.010380,0.002941,0.008389,0.009080
2022-12-29,0.716685,0.955355,0.669198,0.550954,-0.011665,-0.274751,0.021568,-0.073597,-0.073308,-0.073459,-0.019556,-0.011372,-0.009390,-0.008879


In [ ]:
r

,sp15,nepool,paloverde,pjm,spx,gas,oil,term1m,term6m,term1Yr,term5Yr,term7Yr,term10Yr,term20Yr
Date,,,,,,,,,,,,,,
2018-01-02,0.002401,0.010021,0.001237,-0.001735,-0.002511,0.043193,-0.007775,-0.001991,-0.002497,-0.002297,-0.002560,-0.000562,0.000872,0.002383
2018-01-03,0.101075,-0.063223,0.125052,-0.148206,0.006649,0.000776,0.019837,-0.009481,-0.003174,-0.001977,-0.002658,-0.006730,-0.008828,-0.010753
2018-01-04,-0.333518,0.453358,-0.360329,0.751175,0.004550,-0.293345,0.005489,0.007160,0.000429,0.001210,0.008836,0.005259,0.000006,-0.002332
2018-01-05,0.025210,-0.725798,-0.124469,-1.531996,0.006985,0.118992,-0.008439,-0.007294,-0.001370,-0.001345,0.007400,0.008342,0.008846,0.008219
2018-01-06,0.051920,0.281240,0.122001,0.710973,-0.001460,-0.196571,0.001933,0.000750,-0.002110,-0.002014,-0.003805,-0.003330,-0.002736,-0.001862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,-0.047023,-0.007514,-0.050224,-0.222984,0.003369,-0.026433,0.003499,-0.011673,-0.001514,0.000195,0.001701,0.001244,0.001050,0.002620
2022-12-28,0.036134,0.005859,0.036807,0.229624,-0.012469,-0.180384,-0.005596,0.018695,-0.000990,-0.003407,0.006312,0.009248,0.011206,0.006031
2022-12-29,-0.005518,-0.000871,-0.006813,0.003288,0.017877,-0.067886,-0.004369,0.011366,0.000121,0.002149,-0.009176,-0.014314,-0.017779,-0.017959
